In [38]:
import os
import json
import pandas as pd
import traceback

# https://python.langchain.com/docs/integrations/chat/openai
from langchain.chat_models import ChatOpenAI

from dotenv import load_dotenv

load_dotenv()

Python-dotenv could not parse statement starting at line 1


True

In [39]:
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

In [40]:
# Initialize ChatopenAI
llm = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model_name="gpt-3.5-turbo",temperature=0.5)